In [1]:
import os
os.chdir('../')
print(os.getcwd())

/DATA/rahul_1911mt11/multi-tasking/EEG-Multi-Tasking


In [2]:
import pdb
import torch 
import random
import torchvision
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

In [3]:
# Device configuration
gpu = "cuda:1"
device = torch.device(gpu if torch.cuda.is_available() else 'cpu')

# Hyper parameters
episodes = 10
num_epochs = 2
num_classes = 2
batch_size = 50
lr = 0.000001

In [4]:
from src.dataset import multi_task_dataset

In [5]:
df = pd.read_csv('input/data.csv')

In [6]:
df = df.sample(frac=1).reset_index(drop=True)
df.shape

(3480, 5)

In [7]:
def conv_block(in_channels, out_channels):
    '''Convolution Block of 3x3 kernels + batch norm + maxpool of 2x2'''

    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 5, padding=2),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    
def set_grad(model, grad):
    for param in model.parameters():
#         print(param.shape)
        param.requires_grad = grad

In [8]:
class Model_0(nn.Module):
    def __init__(self,):
        super(Model_0, self).__init__()
        
        self.conv1 = conv_block(1, 20)
        self.conv2 = conv_block(20, 40)
        self.conv3 = conv_block(40, 60)
        
        self.conv11 = conv_block(1, 20)
        self.conv22 = conv_block(20, 40)
        self.conv33 = conv_block(40, 60)
        
        self.fc = nn.Linear(9 * 4 * 60 * 2, 100)

    
       #heads
        self.y1o = nn.Linear(100,2)
      
    def forward(self, x):
        
        x1 = self.conv1(x)
        x1 = self.conv2(x1)
        x1 = self.conv3(x1)
        
        x2 = self.conv11(x)
        x2 = self.conv22(x2)
        x2 = self.conv33(x2)

        # Features of shape (batch_size, 64)
        feat1 = x1.reshape(x1.size(0), -1)
        feat2 = x2.reshape(x2.size(0), -1)
        feat  = torch.cat((feat1, feat2),1)
        
        head = self.fc(feat)
  
        # heads
        out = self.y1o(head)
    
        return out

In [9]:
class Model_1(nn.Module):
    def __init__(self, private_layers):
        super(Model_1, self).__init__()
        
        self.x = private_layers
#         self.conv1 = conv_block(1, 20)
#         self.conv2 = conv_block(20, 40)
#         self.conv3 = conv_block(40, 60)
#         self.fc = nn.Linear(9 * 4 * 60 * 2, 100)
#         #heads
#         self.y1o = nn.Linear(100,2)
        
        # Loading shared layers
        s_model = Model_map[0]
        s_model = s_model().to(device)
        try:
            checkpoint = torch.load(f'models/shared_private/model_{0}.pth', map_location=torch.device(gpu if torch.cuda.is_available() else 'cpu'))
            s_model.load_state_dict(checkpoint['weights'])
            
#             print(f'loaded shared layer for model {self.x}')
        except:
#             pdb.set_trace()
            print(f"------------Create a brand new shared layer from scrath -------------------------")
        self.conv11 = s_model.conv11
        self.conv22 = s_model.conv22
        self.conv33 = s_model.conv33
        

        # Loading private layers
        p_model = Model_map[0]
        p_model = p_model().to(device)
        try:
            checkpoint = torch.load(f'models/shared_private/model_{self.x}.pth', map_location=torch.device(gpu if torch.cuda.is_available() else 'cpu'))
            p_model.load_state_dict(checkpoint['weights'])
            
#             print(f'loaded private layer for model {x}')
        except:
#             pdb.set_trace()
#             print(f"------------Create a brand new private layer for subject {self.x} from scrath -------------------------")
            pass
    
        self.conv1 = p_model.conv1
        self.conv2 = p_model.conv2
        self.conv3 = p_model.conv3
        self.fc = p_model.fc
        self.y1o = p_model.y1o
        
    
    
       
      
    def forward(self, x):
        
        x1 = self.conv1(x)
        x1 = self.conv2(x1)
        x1 = self.conv3(x1)
        
        x2 = self.conv11(x)
        x2 = self.conv22(x2)
        x2 = self.conv33(x2)

        # Features of shape (batch_size, 64)
        feat1 = x1.reshape(x1.size(0), -1)
        feat2 = x2.reshape(x2.size(0), -1)
        feat  = torch.cat((feat1, feat2),1)
        
        head = self.fc(feat)
  
        # heads
        out = self.y1o(head)
    
        return out

In [10]:
Model_map = {0:Model_0, 1:Model_1}

In [14]:
# Train the model

x = 0
for episode in range(episodes):
    
#     pdb.set_trace()
    n = random.randint(1,9)
#     n = 1
    df_train = df.loc[(df.Type=='train') & (df.Subject==n)].reset_index(drop = True)
    train_dataset = multi_task_dataset(df_train)

    # Data loader
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                               batch_size=batch_size, 
                                               shuffle=True)
    print(f"Episode : {episode}, n : {n}, x : {x}")
    model = Model_map[1]
    model = model(n).to(device)
#     try:
#         checkpoint = torch.load(f'models/shared_private/model_{n}.pth', map_location=torch.device(gpu if torch.cuda.is_available() else 'cpu'))
#         model.load_state_dict(checkpoint['weights'])
#         model = model.to(device)
#     #         print('-------Trained model loaded-----------')
#     except :
         
#         print(f'-------New  model_{n} created --------------')





    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr)


    total_step = len(train_loader)
    for epoch in range(num_epochs):


        for i, (images, labels, subjects) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device)
            subjects = subjects.to(device)

            # Forward pass 1
            outputs = model(images)
            loss = criterion(outputs, labels)
#             model1.freeze(subjects[0].item())
#             model1.to(device)




            # Backward and optimize 1
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


            torch.save({'weights': model.state_dict()},f'models/shared_private/model_{n}.pth')
            torch.save({'weights': model.state_dict()},f'models/shared_private/model_{0}.pth')
            if (i+1) % 500 == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                       .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

    print(f"-------------  EPISODE : {episode}  ------------")
    print("------------------------------------------------")
    print_training_accuracy()
    print_testing_accuracy()
    print("------------------------------------------------", '\n')
    
    x = n

Episode : 0, n : 3, x : 0
-------------  EPISODE : 0  ------------
------------------------------------------------
Train Accuracy : 94.42028985507247 %
Test Accuracy : 75.97222222222223 %
------------------------------------------------ 

Episode : 1, n : 4, x : 3
-------------  EPISODE : 1  ------------
------------------------------------------------
Train Accuracy : 94.27536231884058 %
Test Accuracy : 76.11111111111111 %
------------------------------------------------ 

Episode : 2, n : 2, x : 4
-------------  EPISODE : 2  ------------
------------------------------------------------
Train Accuracy : 94.27536231884058 %
Test Accuracy : 76.11111111111111 %
------------------------------------------------ 

Episode : 3, n : 4, x : 2
-------------  EPISODE : 3  ------------
------------------------------------------------
Train Accuracy : 94.27536231884058 %
Test Accuracy : 76.11111111111111 %
------------------------------------------------ 

Episode : 4, n : 6, x : 4
------------- 

KeyboardInterrupt: 

In [12]:
def print_training_accuracy():
    df = pd.read_csv('input/data.csv')
    df_train = df.loc[(df.Type=='train') & (df.Subject!=10)].reset_index(drop = True)
    train_dataset = multi_task_dataset(df_train)
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                               batch_size=1, 
                                               shuffle=True)
#     model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
    with torch.no_grad():
        correct = 0
        total = 0
        for image, label, subject in train_loader:
            n = subject[0]
            model = Model_map[1]
            model = model(n).to(device)
            image = image.to(device)
            label = label.to(device)
            outputs = model(image)
            _, predicted = torch.max(outputs, 1)
    #         print('predicted and label = ', predicted.item(), label.item())     

            total += label.size(0)
            correct += (predicted == label).sum().item()
#             print(predicted.item(), label.item())
    # print(total, correct)
    print('Train Accuracy : {} %'.format(100 * correct / total))

In [13]:
# Test the model on test data
def print_testing_accuracy():
    df = pd.read_csv('input/data.csv')
    df_test = df.loc[(df.Type=='test') & (df.Subject!=10)].reset_index(drop = True)
    test_dataset = multi_task_dataset(df_test)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                               batch_size=1, 
                                               shuffle=True)
#     model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
    with torch.no_grad():
        correct = 0
        total = 0
        for image, label, subject in test_loader:
            n = int(subject[0])
            model = Model_map[1]
            model = model(n).to(device)
            image = image.to(device)
            label = label.to(device)
            outputs = model(image)
            _, predicted = torch.max(outputs, 1)
#             print('predicted and label = ', predicted.item(), label.item())     

            total += label.size(0)
            correct += (predicted == label).sum().item()
#             print(predicted.item(), label.item())
#     print(total, correct)
    print('Test Accuracy : {} %'.format(100 * correct / total))